# CHAPTER 5
# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% or more of an analyst’s time. Sometimes the way that data is stored in files or databases is not in the right format for a particular task. Many researchers choose to do ad hoc processing of data from one form to another using a general-purpose programming language, like Python, Perl, R, or Java, or Unix text-processing tools like sed or awk. Fortunately, pandas, along with the built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form. 

## 5.1 Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default. 

The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value **NaN** (Not a Number) to represent missing data. We call this a *sentinel value* that can be easily detected:

In [5]:
import pandas as pd
import numpy as np

In [6]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [7]:
string_data.isnull() 

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as **NA**, which stands for *not available*. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection, for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data. 

The built-in Python None value is also treated as NA in object arrays:

In [8]:
string_data[0] = None
string_data.isnull() 

0     True
1    False
2     True
3    False
dtype: bool

There is work ongoing in the pandas project to improve the internal details of how missing data is handled, but the user API functions, like pandas.isnull, abstract away many of the annoying details. See Table 5-1 for a list of some functions related to missing data handling.

<br>
<center>Table 5.1: NA handling methods</center>
<img src="Table5.1.jpg">

### 5.1.1 Filtering Out Missing Data 

There are a few ways to filter out missing data. While you always have the option to do it by hand using pandas.isnull and boolean indexing, the **dropna** can be helpful. On a Series, it returns the Series with only the non-null data and index values:

In [9]:
from numpy import nan as NA

data = pd.Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [10]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [11]:
data[data.notnull()] 

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame objects, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. **dropna** by default drops any row containing a missing value:

In [12]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],[NA, NA, NA], [NA, 6.5, 3.]])
data 

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,6.5,3.0


Passing **how='all'** will only drop rows that are all NA:

In [14]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass **axis=1**:

In [15]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [16]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the **thresh** argument:

In [17]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df 

,0,1,2
0,-0.434768,NaN,NaN
1,-0.672066,NaN,NaN
2,0.238355,NaN,2.383814
3,-0.765225,NaN,-1.607577
4,-0.020276,-0.075257,-0.815671
5,2.144308,-0.316621,-0.563034
6,-0.678813,-1.364482,-0.410755


In [18]:
df.dropna() 

,0,1,2
4,-0.020276,-0.075257,-0.815671
5,2.144308,-0.316621,-0.563034
6,-0.678813,-1.364482,-0.410755


In [19]:
df.dropna(thresh=2)
# it will drop any rows that contain less than 2 observation values

,0,1,2
2,0.238355,NaN,2.383814
3,-0.765225,NaN,-1.607577
4,-0.020276,-0.075257,-0.815671
5,2.144308,-0.316621,-0.563034
6,-0.678813,-1.364482,-0.410755


### 5.1.2 Filling In Missing Data 

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways. For most purposes, the **fillna** method is the workhorse function to use. Calling **fillna** with a constant replaces missing values with that value:

In [20]:
df.fillna(0)

,0,1,2
0,-0.434768,0.000000,0.000000
1,-0.672066,0.000000,0.000000
2,0.238355,0.000000,2.383814
3,-0.765225,0.000000,-1.607577
4,-0.020276,-0.075257,-0.815671
5,2.144308,-0.316621,-0.563034
6,-0.678813,-1.364482,-0.410755


Calling **fillna** with a dict, you can use a different fill value for each column:

In [21]:
df

,0,1,2
0,-0.434768,NaN,NaN
1,-0.672066,NaN,NaN
2,0.238355,NaN,2.383814
3,-0.765225,NaN,-1.607577
4,-0.020276,-0.075257,-0.815671
5,2.144308,-0.316621,-0.563034
6,-0.678813,-1.364482,-0.410755


In [22]:
df.fillna({1: 0.5, 2: 0}) 

,0,1,2
0,-0.434768,0.500000,0.000000
1,-0.672066,0.500000,0.000000
2,0.238355,0.500000,2.383814
3,-0.765225,0.500000,-1.607577
4,-0.020276,-0.075257,-0.815671
5,2.144308,-0.316621,-0.563034
6,-0.678813,-1.364482,-0.410755


**fillna** returns a new object, but you can modify the existing object *in-place*:

In [23]:
df.fillna(0, inplace=True)
df

,0,1,2
0,-0.434768,0.000000,0.000000
1,-0.672066,0.000000,0.000000
2,0.238355,0.000000,2.383814
3,-0.765225,0.000000,-1.607577
4,-0.020276,-0.075257,-0.815671
5,2.144308,-0.316621,-0.563034
6,-0.678813,-1.364482,-0.410755


The same interpolation methods available for reindexing can be used with **fillna**:

In [24]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df 

,0,1,2
0,-0.693788,-0.054053,0.645281
1,-0.949852,-0.232946,0.119398
2,-0.991162,NaN,-2.884015
3,0.580824,NaN,0.741954
4,-2.216122,NaN,NaN
5,-0.498152,NaN,NaN


In [25]:
df.fillna(method='ffill')

C:\Users\Izzah Alia\AppData\Local\Temp\ipykernel_1876\1193302488.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill')


,0,1,2
0,-0.693788,-0.054053,0.645281
1,-0.949852,-0.232946,0.119398
2,-0.991162,-0.232946,-2.884015
3,0.580824,-0.232946,0.741954
4,-2.216122,-0.232946,0.741954
5,-0.498152,-0.232946,0.741954


In [26]:
 df.fillna(method='ffill', limit=2) 

C:\Users\Izzah Alia\AppData\Local\Temp\ipykernel_1876\531624609.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', limit=2)


,0,1,2
0,-0.693788,-0.054053,0.645281
1,-0.949852,-0.232946,0.119398
2,-0.991162,-0.232946,-2.884015
3,0.580824,-0.232946,0.741954
4,-2.216122,NaN,0.741954
5,-0.498152,NaN,0.741954


With **fillna** you can do lots of other things with a little creativity. For example, you might pass the *mean* or *median* value of a Series:

In [27]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [28]:
data.fillna(data.mean()) 

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

See Table 5-2 for a reference on **fillna**.

<br>
<center>Table 5.2:  fillna function arguments </center>
<img src="Table5.2.jpg">

In [29]:
https://chrisalbon.com/python/data_wrangling/pandas_missing_data/

SyntaxError: invalid syntax (3545541643.py, line 1)

## 5.2 Data Transformation 

So far in this chapter we’ve been concerned with rearranging data. Filtering, cleaning, and other transformations are another class of important operations. 

### 5.2.1 Removing Duplicates 

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [30]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method **duplicated** returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [31]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, **drop_duplicates** returns a DataFrame where the duplicated array is False:

In [32]:
data.drop_duplicates() 

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [33]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [34]:
data.drop_duplicates(['k1']) 

,k1,k2,v1
0,one,1,0
1,two,1,1


**duplicated** and **drop_duplicates** by default keep the first observed value combination. Passing *keep='last'* will return the last one:

In [35]:
data.drop_duplicates(['k1', 'k2'], keep='last') 

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 5.2.2 Transforming Data Using a Function or Mapping 

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of vegetables:

In [36]:
data = pd.DataFrame({'vege': ['cabbage', 'carrot', 'cabbage','Lettuce', 'potato', 'Cabbage', 
                             'lettuce', 'tomato', 'cucumber'],  'weight': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,vege,weight
0,cabbage,4.0
1,carrot,3.0
2,cabbage,12.0
3,Lettuce,6.0
4,potato,7.5
5,Cabbage,8.0
6,lettuce,3.0
7,tomato,5.0
8,cucumber,6.0


Suppose you wanted to add a column indicating the seller that each food came from. Let’s write down a mapping of each distinct vegetables type to the seller:

In [37]:
vege_to_seller = {'cabbage': 'David',  'carrot': 'David',  'lettuce': 'Ahmad',  'potato': 'Ahmad', 
                  'tomato': 'David',  'cucumber': 'Tina'}

The **map** method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the vegetables are capitalized and others are not. Thus, we need to convert each value to lowercase using the **str.lower** Series method:

In [38]:
lowercased = data['vege'].str.lower()
lowercased

0     cabbage
1      carrot
2     cabbage
3     lettuce
4      potato
5     cabbage
6     lettuce
7      tomato
8    cucumber
Name: vege, dtype: object

In [39]:
data['seller'] = lowercased.map(vege_to_seller)
data

,vege,weight,seller
0,cabbage,4.0,David
1,carrot,3.0,David
2,cabbage,12.0,David
3,Lettuce,6.0,Ahmad
4,potato,7.5,Ahmad
5,Cabbage,8.0,David
6,lettuce,3.0,Ahmad
7,tomato,5.0,David
8,cucumber,6.0,Tina


We could also have passed a function that does all the work:

In [40]:
data['vege'].map(lambda x: vege_to_seller[x.lower()]) 

0    David
1    David
2    David
3    Ahmad
4    Ahmad
5    David
6    Ahmad
7    David
8     Tina
Name: vege, dtype: object

Using **map** is a convenient way to perform element-wise transformations and other data cleaning–related operations. 

### 5.2.3 Replacing Values 

Filling in missing data with the **fillna** method is a special case of more general value replacement. As you’ve already seen, **map** can be used to modify a subset of values in an object but **replace** provides a simpler and more flexible way to do so. Let’s consider this Series:

In [41]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA values that pandas understands, we can use replace, producing a new Series (unless you pass inplace=True):

In [42]:
data.replace(-999, np.nan) 

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the substitute value:

In [43]:
data.replace([-999, -1000], np.nan) 

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [44]:
data.replace([-999, -1000], [np.nan, 0]) 

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:

In [45]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The **data.replace** method is distinct from **data.str.replace**, which performs string substitution element-wise. We look at these string methods on Series later in the chapter.

### 5.2.4 Renaming Axis Indexes 

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [46]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'], 
                    columns=['one', 'two', 'three', 'four']) 
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Like a Series, the axis indexes have a **map** method:

In [47]:
transform = lambda x: x[:4].upper()
data.index.map(transform) 

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

You can assign to **index**, modifying the DataFrame in-place:

In [48]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the original, a useful method is **rename**:

In [49]:
data.rename(index=str.title, columns=str.upper) 

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, **rename** can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [50]:
 data.rename(index={'OHIO': 'INDIANA'},columns={'three': 'peekaboo'}) 

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


**rename** saves you from the chore of copying the DataFrame manually and assigning to its **index** and **columns** attributes. Should you wish to modify a dataset in-place, pass *inplace=True*:

In [54]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 5.2.5 Discretization and Binning 

Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [55]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32] 

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do so, you have to use **cut**, a function in pandas:

In [56]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special **Categorical** object. The output you see describes the bins computed by **pandas.cut**. You can treat it like an array of strings indicating the bin name; internally it contains a **categories** array specifying the distinct category names along with a labeling for the **ages** data in the codes attribute:

In [57]:
cats.codes 

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [58]:
cats.categories 

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [59]:
pd.value_counts(cats) 

C:\Users\Izzah Alia\AppData\Local\Temp\ipykernel_1876\2346239419.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(cats)


(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Note that **pd.value_counts(cats)** are the bin counts for the result of **pandas.cut**. 

Consistent with mathematical notation for intervals, a parenthesis means that the side is open, while the square bracket means it is closed (inclusive). You can change which side is closed by passing right=False:

In [60]:
 pd.cut(ages, [18, 26, 36, 61, 100], right=False) 

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the **labels** option:

In [61]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names) 

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to **cut** instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [62]:
data = np.random.rand(20)
data

array([0.2751211 , 0.31068762, 0.02120308, 0.79336946, 0.8180004 ,
       0.08238861, 0.19212681, 0.79406326, 0.14887576, 0.15752412,
       0.63034065, 0.52209567, 0.53126922, 0.48526594, 0.79073581,
       0.00117599, 0.71565719, 0.1398577 , 0.40605609, 0.4033652 ])

In [63]:
pd.cut(data, 4, precision=2) 

[(0.21, 0.41], (0.21, 0.41], (0.00036, 0.21], (0.61, 0.82], (0.61, 0.82], ..., (0.00036, 0.21], (0.61, 0.82], (0.00036, 0.21], (0.21, 0.41], (0.21, 0.41]]
Length: 20
Categories (4, interval[float64, right]): [(0.00036, 0.21] < (0.21, 0.41] < (0.41, 0.61] < (0.61, 0.82]]

The *precision=2* option limits the decimal precision to two digits. 

A closely related function, **qcut**, bins the data based on sample quantiles. Depending on the distribution of the data, using **cut** will not usually result in each bin having the same number of data points. Since **qcut** uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [64]:
data = np.random.randn(1000)  # Normally distributed
data

array([-1.00451553e+00,  1.34161675e+00, -1.09673058e+00, -3.04316540e-01,
       -5.79181873e-01, -2.51359217e-01, -4.39946996e-01,  1.05361321e-02,
        1.92675623e-01, -1.01958367e-01, -5.00919562e-01,  2.56592952e-01,
       -6.85621659e-01,  5.63587345e-01,  2.61991923e-01,  1.03164982e+00,
       -1.10916343e+00,  7.78224264e-01,  1.46055539e-01, -6.84716196e-01,
        3.77734152e-01, -8.59828328e-01,  9.81543367e-01, -4.60642903e-01,
        3.03300274e-01,  7.48125839e-02, -8.04340292e-01,  6.09378294e-01,
        3.77433153e-01,  2.60849061e-01, -5.38070066e-01,  1.01714596e+00,
       -7.12274350e-01,  1.72119184e+00,  2.59559035e-01, -6.33687602e-01,
       -1.53642848e-01,  6.34772685e-01, -5.67581907e-01,  4.13832294e-01,
        5.00027411e-01,  1.07796648e+00, -1.54765363e+00,  2.14584061e+00,
       -1.31304499e+00, -6.98601605e-01,  1.19393620e+00,  9.30673755e-01,
        2.66122238e+00, -1.76311521e-01, -1.78139958e+00,  1.26902701e-01,
       -1.12692120e+00, -

In [65]:
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(-2.896, -0.685], (0.647, 2.846], (-2.896, -0.685], (-0.685, -0.00914], (-0.685, -0.00914], ..., (-0.00914, 0.647], (-0.00914, 0.647], (0.647, 2.846], (-0.685, -0.00914], (-2.896, -0.685]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.896, -0.685] < (-0.685, -0.00914] < (-0.00914, 0.647] < (0.647, 2.846]]

In [71]:
pd.value_counts(cats) 

C:\Users\Izzah Alia\AppData\Local\Temp\ipykernel_1876\2346239419.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(cats)


(-2.896, -0.685]      250
(-0.685, -0.00914]    250
(-0.00914, 0.647]     250
(0.647, 2.846]        250
Name: count, dtype: int64

Similar to **cut** you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [72]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) 

[(-1.188, -0.00914], (1.283, 2.846], (-1.188, -0.00914], (-1.188, -0.00914], (-1.188, -0.00914], ..., (-0.00914, 1.283], (-0.00914, 1.283], (1.283, 2.846], (-1.188, -0.00914], (-1.188, -0.00914]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.896, -1.188] < (-1.188, -0.00914] < (-0.00914, 1.283] < (1.283, 2.846]]

## 5.3  String Manipulation 

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data. 


### 5.3.1 String Object Methods 

In many string munging and scripting applications, built-in string methods are sufficient. As an example, a comma-separated string can be broken into pieces with **split**:

In [74]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

**split** is often combined with **strip** to trim whitespace (including line breaks):

In [75]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

These substrings could be concatenated together with a two-colon delimiter using addition:

In [76]:
first, second, third = pieces
first + '::' + second + '::' + third 

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the **join** method on the string '::':

In [77]:
'::'.join(pieces) 

'a::b::guido'

Other methods are concerned with locating substrings. Using Python’s **in** keyword is the best way to detect a substring, though **index** and **find* can also be used:

In [78]:
'guido' in val 

True

In [79]:
val.index(',') 

1

In [80]:
val.find(':') 

-1

Note the difference between **find** and **index** is that index raises an exception if the string isn’t found (versus returning –1):

In [81]:
val.index(':') 

ValueError: substring not found

Relatedly, **count** returns the number of occurrences of a particular substring:

In [82]:
val.count(',') 

2

**replace** will substitute occurrences of one pattern for another. It is commonly used to delete patterns, too, by passing an empty string:


In [83]:
val.replace(',', '::') 

'a::b::  guido'

In [84]:
val.replace(',', '') 

'ab  guido'

See Table 5-3 for a listing of some of Python’s string methods. Regular expressions can also be used with many of these operations, as you’ll see.

<br>
<center>Table 5.3: Python built-in string methods</center>
<img src="Table5.3.jpg">

### 5.3.2  Regular Expressions 

*Regular expressions* provide a flexible way to search or match (often more complex) string patterns in text. A single expression, commonly called a *regex*, is a string formed according to the regular expression language. Python’s built-in **re** module is responsible for applying regular expressions to strings.

The **re** module functions fall into three categories: pattern matching, substitution, and splitting. Naturally these are all related; a *regex* describes a pattern to locate in the text, which can then be used for many purposes. Let’s look at a simple example:

Suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines). The *regex* describing one or more whitespace characters is **\s+**:

In [86]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text) 

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Izzah Alia\AppData\Local\Temp\ipykernel_1876\2948975259.py:3: SyntaxWarning: invalid escape sequence '\s'
  re.split('\s+', text)


['foo', 'bar', 'baz', 'qux']

When you call **re.split('\s+', text)**, the regular expression is first compiled, and then its split method is called on the passed text. You can compile the *regex* yourself with **re.compile**, forming a reusable *regex* object:

In [87]:
regex = re.compile('\s+')
regex.split(text) 

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Izzah Alia\AppData\Local\Temp\ipykernel_1876\2791392083.py:1: SyntaxWarning: invalid escape sequence '\s'
  regex = re.compile('\s+')


['foo', 'bar', 'baz', 'qux']

If, instead, you wanted to get a list of all patterns matching the *regex*, you can use the **findall** method:

In [88]:
regex.findall(text) 

['    ', '\t ', '  \t']

**match** and **search** are closely related to **findall**. While **findall** returns all matches in a string, **search** returns only the first match. More rigidly, **match** only matches at the beginning of the string. As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:

In [89]:
text = """Dave dave@google.com 
Steve steve@gmail.com 
Rob rob@gmail.com 
Ryan ryan@yahoo.com 
""" 
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

#re.IGNORECASE makes the regex case-insensitive 
regex = re.compile(pattern, flags=re.IGNORECASE) 

Using **findall** on the text produces a list of the email addresses:

In [90]:
regex.findall(text) 

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

**search** returns a special match object for the first email address in the text. For the preceding *regex*, the match object can only tell us the start and end position of the pattern in the string:

In [91]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [92]:
text[m.start():m.end()] 

'dave@google.com'

**regex.match** returns None, as it only will match if the pattern occurs at the start of the string:

In [93]:
print(regex.match(text)) 

None


Relatedly, **sub** will return a new string with occurrences of the pattern replaced by the a new string:

In [94]:
print(regex.sub('REDACTED', text)) 

Dave REDACTED 
Steve REDACTED 
Rob REDACTED 
Ryan REDACTED 



Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix. To do this, put parentheses around the parts of the pattern to segment:

In [95]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE) 

A match object produced by this modified *regex* returns a tuple of the pattern components with its **groups** method:

In [96]:
m = regex.match('wesm@bright.net')
m.groups() 

('wesm', 'bright', 'net')

**findall** returns a list of tuples when the pattern has groups:

In [97]:
regex.findall(text) 

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

**sub** also has access to groups in each match using special symbols like \1 and \2. The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:

In [98]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text)) 

Dave Username: dave, Domain: google, Suffix: com 
Steve Username: steve, Domain: gmail, Suffix: com 
Rob Username: rob, Domain: gmail, Suffix: com 
Ryan Username: ryan, Domain: yahoo, Suffix: com 



There is much more to regular expressions in Python, most of which is outside our scope. Table 5-4 provides a brief summary.

<br>
<center>Table 5.4: Regular expression methods</center>
<img src="Table5.4.jpg">

### 5.3.3 Vectorized String Functions in pandas 

Cleaning up a messy dataset for analysis often requires a lot of string munging and regularization. To complicate matters, a column containing strings will sometimes have missing data:

In [99]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [100]:
data.isnull() 

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

String and regular expression methods can be applied (passing a **lambda** or other function) to each value using **data.map**, but it will fail on the NA (null) values. To cope with this, Series has array-oriented methods for string operations that skip NA values. These are accessed through Series’s **str** attribute; for example, we could check whether each email address has 'gmail' in it with **str.contains**:

In [101]:
data.str.contains('gmail') 

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Regular expressions can be used, too, along with any **re** options like **IGNORECASE**:

In [102]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [103]:
data.str.findall(pattern, flags=re.IGNORECASE) 

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use **str.get** or index into the **str** attribute:

In [104]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

To access elements in the embedded lists, we can pass an index to either of these functions:

In [105]:
matches.str.get(1) 

AttributeError: Can only use .str accessor with string values!

In [ ]:
matches.str[0] 

You can similarly slice strings using this syntax:

In [ ]:
data.str[:5] 

See Table 5-5 for more pandas string methods.

<br>
<center>Table 5.5: Partial listing of vectorized string methods</center>
<img src="Table5.5.jpg">